In [ ]:
!pip3 install sentence-transformers chromadb pandas numpy
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q chromadb sentence-transformers pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.8 MB/s eta

In [ ]:
from google.colab import files

uploaded = files.upload()



Saving data.xlsx to data.xlsx


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving Forcast.xlsx to Forcast.xlsx


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings

import os

In [ ]:
df1 = pd.read_excel("data.xlsx")
df2 = pd.read_excel("Forcast.xlsx")

In [ ]:
def dataframe_to_text_chunks(df, table_name: str, max_chunk_chars: int = 1000) -> list:
    """
    Converts DataFrame rows into text chunks for embedding.
    Splits long rows into multiple smaller chunks to avoid token overflows.
    """
    chunks = []
    for _, row in df.iterrows():
        parts = [f"{col}: {str(row[col])}" for col in df.columns if pd.notnull(row[col])]
        full_text = f"[{table_name}] " + "; ".join(parts)

        if len(full_text) <= max_chunk_chars:
            chunks.append(full_text)
        else:
            current = f"[{table_name}] "
            for part in parts:
                if len(current) + len(part) + 2 <= max_chunk_chars:
                    current += part + "; "
                else:
                    chunks.append(current.strip("; "))
                    current = f"[{table_name}] {part}; "
            if current:
                chunks.append(current.strip("; "))
    return chunks

chunks1 = dataframe_to_text_chunks(df1, "Data")
chunks2 = dataframe_to_text_chunks(df2, "Forcast")
all_chunks = chunks1 + chunks2

print(f"✅ Total optimized chunks: {len(all_chunks)}")

✅ Total optimized chunks: 678194


In [ ]:
import torch
from sentence_transformers import SentenceTransformer

# Detect GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔍 Using device: {device}")

# Load model on GPU if available
model = SentenceTransformer("all-MiniLM-L6-v2", device=device)

# Embedding step
print("🔄 Embedding...")
embeddings = model.encode(
    all_chunks,
    convert_to_numpy=True,
    show_progress_bar=True,
    device=device  # This ensures the actual computation is on GPU
)
print("✅ Embedding completed.")


🔍 Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔄 Embedding...


Batches:   0%|          | 0/21194 [00:00<?, ?it/s]

✅ Embedding completed.


In [ ]:
import shutil
import os

chroma_path = "/content/chroma_rag_db"
if os.path.exists(chroma_path):
    shutil.rmtree(chroma_path)
os.makedirs(chroma_path, exist_ok=True)

In [ ]:
import chromadb
from chromadb.config import Settings, DEFAULT_TENANT, DEFAULT_DATABASE

client = chromadb.PersistentClient(
    path=chroma_path,
    settings=Settings(),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

collection = client.get_or_create_collection(name="excel_chunks")

In [ ]:
print("💾 Saving to ChromaDB...")
batch_size = 5000
for i in range(0, len(all_chunks), batch_size):
    end = min(i + batch_size, len(all_chunks))
    collection.add(
        documents=all_chunks[i:end],
        embeddings=embeddings[i:end],
        ids=[f"doc_{j}" for j in range(i, end)]
    )

print(f"✅ ChromaDB saved at: {chroma_path}")

💾 Saving to ChromaDB...
✅ ChromaDB saved at: /content/chroma_rag_db


In [ ]:
!zip -r chroma_rag_db.zip chroma_rag_db
files.download("chroma_rag_db.zip")

  adding: chroma_rag_db/ (stored 0%)
  adding: chroma_rag_db/chroma.sqlite3 (deflated 79%)
  adding: chroma_rag_db/fd440ea9-91fe-4b35-971d-1e93aa25d767/ (stored 0%)
  adding: chroma_rag_db/fd440ea9-91fe-4b35-971d-1e93aa25d767/link_lists.bin (deflated 75%)
  adding: chroma_rag_db/fd440ea9-91fe-4b35-971d-1e93aa25d767/header.bin (deflated 53%)
  adding: chroma_rag_db/fd440ea9-91fe-4b35-971d-1e93aa25d767/data_level0.bin (deflated 13%)
  adding: chroma_rag_db/fd440ea9-91fe-4b35-971d-1e93aa25d767/index_metadata.pickle (deflated 64%)
  adding: chroma_rag_db/fd440ea9-91fe-4b35-971d-1e93aa25d767/length.bin (deflated 81%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>